In [ ]:
from hackathon_utils.my_imports import *
from hackathon_utils.reproducibility import set_seed
from hackathon_utils.auxiliar import *
from hackathon_utils.datasets import get_dataloaders
from hackathon_utils.data_config import *
from hackathon_utils.loggers import get_logger
from hackathon_utils.training_config import *
from hackathon_utils.training import *
from hackathon_utils.presentation import segment_images
import segmentation_models_pytorch as smp
import timm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
set_seed()
logger = get_logger()

# Ensure output folder exists before opening the CSV file
create_folder_if_not_exists(OUTPUT_FOLDER_NAME)

try:
    # --- Build data config / preprocessing from a timm backbone (feature extractor only) ---
    logger.info("Loading backbone for data config...")
    timm_backbone_for_cfg = timm.create_model(
        BACKBONE_NAME, num_classes=0, pretrained=True
    )
    data_config = timm.data.resolve_model_data_config(timm_backbone_for_cfg)
    logger.info(f"Data config: {data_config}")

    preprocessing_train = timm.data.create_transform(
        **data_config, is_training=True
    )
    preprocessing_test = timm.data.create_transform(
        **data_config, is_training=False
    )

    # --- Dataloaders ---
    train_loader, val_loader = get_dataloaders(
        data_config
    )
    logger.info(
        f"{len(train_loader.dataset)}, {len(val_loader.dataset)}"
    )

    # --- Model build + training ---
    try:
        logger.info("Creating segmentation model...")
        model = smp.create_model(
            ARCHITECTURE_NAME,
            encoder_name=f"tu-{BACKBONE_NAME}",
            in_channels=3,
            encoder_weights="imagenet",
            classes=NUM_CLASSES,
        ).to(device)

        total_params, trainable_params = count_params(model)
        logger.info(
            f"Params — total: {total_params/1e6:.3f}M | "
            f"trainable: {trainable_params/1e6:.3f}M"
        )

        (
            best_train_loss,
            best_train_jaccard_score,
            best_val_loss,
            best_val_jaccard_score,
            best_epoch,
            losses,
            best_train_jaccard_per_class,
            best_val_jaccard_per_class,
        ) = train_model(
            model,
            train_loader,
            val_loader,
            max_lr=LEARNING_RATE,
            weight_decay=WEIGHT_DECAY,
        )

        segment_images(
            model,
            input_folder=TEST_IMG_DIR,
            output_folder=RESULTS_IMG_DIR,
            device=device
        )

    except Exception as e:
        logger.error(f"Training block failed: {e}")
        logger.error(traceback.format_exc())

except IOError as e:
    logger.error(f"Error writing to file '{FILENAME}': {e}")
except Exception as e:
    logger.error("Unexpected error at top-level try/with block.")
    logger.error(traceback.format_exc())